In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

La data corresponde a la serie por condado de covid en el estado de Carolina del Norte, el objetivo es generar una visualización interactiva donde la serie de tiempo de casos nuevos interactue con el mapa de los condados

In [ ]:
data= pd.read_csv('datanc.csv')
data

Al trabajar con series de tiempo, es importante mantener el orden en el que se presentan los datos es determinante, se recomienda usar los tipos de datos proveidos por pandas

In [ ]:
data['Date']=pd.to_datetime(data['Date']).dt.date

In [ ]:
new_cases=data.pivot('County', columns= 'Date',values='New cases' )
new_cases=new_cases.fillna(0)
new_cases=new_cases.rename(columns=dict(zip(new_cases.columns, [i.strftime('%Y-%m-%d') for i in new_cases.columns])))
new_cases

In [ ]:
new_cases_t=data.pivot('Date', columns= 'County',values='New cases')
new_cases_t=new_cases_t.fillna(0)
new_cases_t.head(5)

In [ ]:
new_cases_t.index

Para la construcción de joins como siempre, es relevante tomar en cuenta cualquier posible asimetría en los registros entre ambas tablas

In [ ]:
ncshp=gpd.read_file('counties.shp')
ncshp['CO_NAME']=ncshp.CO_NAME.str.lower().str.capitalize()
counties= ncshp[['CO_NAME', 'geometry']]

In [ ]:
ncshp

In [ ]:
new_cases

In [ ]:
new_cases[~new_cases.index.isin(counties.CO_NAME.tolist())]

In [ ]:
counties['CO_NAME'].iloc[47]='McDowell'
counties['CO_NAME'].iloc[92]='New Hanover'

In [ ]:
new_cases_shp=gpd.GeoDataFrame(new_cases.join(counties.set_index('CO_NAME')), crs= ncshp.crs, geometry='geometry')
new_cases_shp.plot()

In [ ]:
new_cases_shp.crs

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
import json
from dash.dependencies import Input, Output

In [ ]:
counties

Ejemplos de visualizaciones en dash en https://dash-gallery.plotly.host/Portal/ 

Las geometrías deben ser en algunos casos presentadas en el formato geojson para las funciones de plotly y de dash

In [ ]:
counties.set_index('CO_NAME')
counties=json.loads(counties.set_index('CO_NAME').to_crs(4326).to_json())

In [ ]:
counties

In [ ]:
new_cases_shp.columns

In [ ]:
new_cases.columns

La función choropleth_mapbox permite generar la base de mapas con algunos componentes de interacción, las geometrías son inferidas desde el geojson.

In [ ]:
ncmap = px.choropleth_mapbox(new_cases.reset_index(), geojson=counties, locations= 'County', 
                             color='2020-09-18',
                           color_continuous_scale="Viridis",
                           range_color=(0,max(new_cases_shp['2020-09-18'])),
                           mapbox_style="carto-positron",
                           center={'lon':-80.793457,'lat':35.782169},
                           zoom=5.5,
                           labels={'2020-09-18':'New cases'}
                          )
ncmap.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

ncmap.show()

Tanto plotly como pandas cuentan con componentes que permiten añadir interactívidad a las visualizaciones, en este caso se muestra una opción de componente (dropdown).

Dash genera una aplicación web en la que vivirá la visualización y sus distintos componentes. Al ser un entorno web, se estructura a traves de HTML como también se pueden incorporar elementos estilísticos asociados a CSS.

La interactividad con componentes o entre gráficos se verán mas adelante

Para instanciar la aplicación se genera un objeto de la clase Dash de la libreria dash

In [ ]:
labels= []
for i in new_cases_shp.index:
    labels.append({'label': i, 'value':i})
app2 = dash.Dash()
app2.layout=html.Div([
    
    html.H1('NC Covid', style={'text-align': 'center'}),
    
    dcc.Dropdown(id= "County",
                options= labels,
                multi= True,
                value= 'Alamance',
                style= {'width': '40%'}),
    
    html.Div(id= 'output_container', children= []),
    html.Br(),
    
    dcc.Graph(id= 'my_map', figure= ncmap)
    
    
])

#app2.run_server(debug=True, use_reloader=False)


Muestra de visualización de series de tiempo y añadir botones de filtro 

In [ ]:


newcts = px.line(new_cases_t.reset_index(), x="Date", y=new_cases_t.columns,
              
              title='Total cases over time')

newcts.show()

En la visualización offline, se pueden generar interacciones sencillas, en este caso se muestra un boton de filtro que se actualiza la visualización mediante un diccionario asociado a cada a valor según el cual se filtra

In [ ]:
# # plotly
newcts = go.Figure()

# set up ONE trace
newcts.add_trace(go.Scatter(x=new_cases_t.index,
                         y=new_cases_t[new_cases_t.columns[0]],
                         visible=True)
                )

updatemenu = []
buttons = []

# button with one option for each dataframe
for col in new_cases_t.columns:
    buttons.append(dict(method='restyle',
                        label=col,
                        visible=True,
                        args=[{'y':[new_cases_t[col]],
                               'x':[new_cases_t.index],
                               'type':'scatter'}, [0]],
                        )
                  )

# some adjustments to the updatemenus
                 
updatemenu = []
your_menu = dict()
updatemenu.append(your_menu)

updatemenu[0]['buttons'] = buttons
updatemenu[0]['direction'] = 'down'
updatemenu[0]['showactive'] = True

# add dropdown menus to the figure
newcts.update_layout(showlegend=False, updatemenus=updatemenu)
newcts.show()


In [ ]:
new_cases

Es relevante notar, que se presentan dos herramientas de visualiza, Graph objects y Plotly express que fue mostrada anteriormente, se recomienda ver las diferencias en la documentación

In [ ]:
fig = go.Figure(go.Choroplethmapbox(geojson=counties, locations= new_cases.index,
                                    z=new_cases['2020-09-18'],
                                    colorscale="Viridis", zmin=0, zmax=max(new_cases['2020-09-18']),
                                    marker_opacity=0.5, marker_line_width=0))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=5.75, mapbox_center = {"lat": 35.782169, "lon": -80.793457})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Los objetos graficos se pueden pasar como un diccionario en el parámetro figure de la funcion graph de dash core components, esto permite un desarrollo intuitivo dentro del contexto de dash (dentro de DIVS html, como también se es textual con los elementos sobre los cuales se pueden hacer callbacks

In [ ]:
#https://plotly.com/python/mapbox-layers/

app = dash.Dash()
app.layout=html.Div([dcc.Graph(id='Graph',
                              figure= {
                                  'data':[go.Choroplethmapbox(geojson=counties,
                                                              locations= new_cases.reset_index()['County'],
                                                              z=new_cases['2020-09-18'],
                                                              colorscale="Viridis", zmin=0, zmax=max(new_cases['2020-09-18']),
                                                              marker_opacity=0.5, marker_line_width=0
                                                              )],
                                  'layout':go.Layout(mapbox_style='carto-darkmatter',
                                                    mapbox_zoom=5.75,
                                                    mapbox_center = {"lat": 35.782169, "lon": -80.793457},
                                                    margin={"r":0,"t":0,"l":0,"b":0})
                                                     
                              })
    
])

#app.run_server(debug=True, use_reloader=False)


En Dash se puede de manera sencilla incorporar mas de una visualización en distintos bloques del código html generado

In [ ]:
app = dash.Dash()
app.layout=html.Div([html.Div([dcc.Graph(id='Map',
                              figure= {
                                  'data':[go.Choroplethmapbox(geojson=counties,
                                                              locations= new_cases.reset_index()['County'],
                                                              z=new_cases['2020-09-18'],
                                                              colorscale="Viridis", zmin=0, zmax=max(new_cases['2020-09-18']),
                                                              marker_opacity=0.5, marker_line_width=0
                                                              )],
                                  'layout':go.Layout(mapbox_style='carto-darkmatter',
                                                    mapbox_zoom=5.75,
                                                    mapbox_center = {"lat": 35.782169, "lon": -80.793457},
                                                    margin={"r":0,"t":0,"l":0,"b":0})
                              })], style={'width': '50%', 'display': 'inline-block'}
                               
                              ),
                     
                     html.Div([dcc.Graph(id= 'TS',
                               figure= {
                                   'data': [go.Scatter(x=new_cases_t.index,
                                                       y=new_cases_t[new_cases_t.columns[0]],
                                                       visible=True)]}
                                        )])
                    ])
#app.run_server()

Para generar interacción con el usuario dentro de la aplicación, se utilizarán los callbacks que se declaran a traves del decorador app.callback, este toma como parametro output el elemento que será cambiado y como input el elemento con el cual interactua

In [ ]:
app = dash.Dash()
app.layout=html.Div([html.Div([dcc.Graph(id='Map',
                              figure= {
                                  'data':[go.Choroplethmapbox(geojson=counties,
                                                              locations= new_cases.reset_index()['County'],
                                                              z=new_cases['2020-09-18'],
                                                              colorscale="Viridis", 
                                                              zmin=0, 
                                                              zmax=max(new_cases['2020-09-18']),
                                                              marker_opacity=0.5,
                                                              marker_line_width=0
                                                              )],
                                  'layout':go.Layout(mapbox_style='carto-darkmatter',
                                                    mapbox_zoom=5.75,
                                                    mapbox_center = {"lat": 35.782169, "lon": -80.793457},
                                                     hovermode= 'closest',
                                                    margin={"r":0,"t":0,"l":0,"b":0})
                              })], style={'width': '75%', 'display': 'inline-block'}
                               
                              ),
                     
                     html.Div([dcc.Graph(id= 'TS',
                               figure= {
                                   'data': [go.Scatter(x=new_cases_t.index,
                                                       y=new_cases_t[new_cases_t.columns[0]],
                                                       visible=True)]}
                                        )]),
                     html.Div(html.Pre(id= 'hover-data', style={'paddingTop': 35}))
                    ])
@app.callback(Output('hover-data', 'children'),
             [Input('Map', 'clickData')])
def callback_json(hoverData):
    return json.dumps(hoverData, indent=2)

#app.run_server()

Evidentemente, un aplicación puede contener varios callbacks y permite generar la interacción entre distintos gráficos

In [ ]:
app = dash.Dash()
app.layout=html.Div([html.Div([dcc.Graph(id='Map',
                              figure= {
                                  'data':[go.Choroplethmapbox(geojson=counties,
                                                              locations= new_cases.reset_index()['County'],
                                                              z=new_cases['2020-09-18'],
                                                              colorscale="Viridis", 
                                                              zmin=0, 
                                                              zmax=max(new_cases['2020-09-18']),
                                                              marker_opacity=0.5,
                                                              marker_line_width=0
                                                              )],
                                  'layout':go.Layout(mapbox_style='carto-darkmatter',
                                                    mapbox_zoom=5.75,
                                                    mapbox_center = {"lat": 35.782169, "lon": -80.793457},
                                                     hovermode= 'closest',
                                                    margin={"r":0,"t":0,"l":0,"b":0})
                              })], style={'width': '75%', 'display': 'inline-block'}
                               
                              ),
                     
                     html.Div([dcc.Graph(id= 'TS',
                               figure= {
                                   'data': [go.Scatter(x=new_cases_t.index,
                                                       y=new_cases_t[new_cases_t.columns[0]],
                                                       visible=True)]}
                                        )]),
                     html.Div(html.Pre(id= 'hover-data', style={'paddingTop': 35}))
                    ])
@app.callback(Output('hover-data', 'children'),
             [Input('Map', 'clickData')])
def callback_json(hoverData):
    return json.dumps(hoverData, indent=2)

@app.callback(Output('TS', 'figure'),
             [Input('Map', 'clickData')])
def callback_map(hoverData):
    location= hoverData['points'][0]['location']
    figure= {'data': [go.Scatter(x=new_cases_t.index,
                                 y=new_cases_t[location],
                                 visible=True)],
            'layout': go.Layout(title= '{0} new cases'.format(location))}
    return figure
    

#app.run_server()

La librería dash cuenta con una cantidad relevante de componentes html prediseñados que se pueden ver en la documentación

In [ ]:
#https://dash.plotly.com/dash-core-components

app = dash.Dash()
app.layout=html.Div([html.Div([dcc.Graph(id='Map',
                              figure= {
                                  'data':[go.Choroplethmapbox(geojson=counties,
                                                              locations= new_cases.reset_index()['County'],
                                                              z=new_cases['2020-09-18'],
                                                              colorscale="Viridis", 
                                                              zmin=0, 
                                                              zmax=max(new_cases['2020-09-18']),
                                                              marker_opacity=0.5,
                                                              marker_line_width=0
                                                              )],
                                  'layout':go.Layout(mapbox_style='carto-darkmatter',
                                                    mapbox_zoom=5.75,
                                                    mapbox_center = {"lat": 35.782169, "lon": -80.793457},
                                                     hovermode= 'closest',
                                                    margin={"r":0,"t":0,"l":0,"b":0})
                              })], style={'width': '75%', 'display': 'inline-block'}
                               
                              ),
                     html.Div(dcc.RangeSlider(id= 'slider',
                                              updatemode = 'mouseup',
                                             min=0,
                                             max=len(new_cases_t.index.unique()),
                                             value=[0,
                                                    len(new_cases_t.index.unique())-1],
                                             )
                              
                              ),
                     html.Div(id='output-container-range-slider'),
                     
                     html.Div([dcc.Graph(id= 'TS',
                               figure= {
                                   'data': [go.Scatter(x=new_cases_t.index,
                                                       y=new_cases_t[new_cases_t.columns[0]],
                                                       visible=True)]}
                                        )])
                    ])


@app.callback(Output('TS', 'figure'),
             [Input('Map', 'clickData')])
def callback_map(hoverData):
    location= hoverData['points'][0]['location']
    figure= {'data': [go.Scatter(x=new_cases_t.index,
                                 y=new_cases_t[location],
                                 visible=True)],
            'layout': go.Layout(title= '{0} new cases'.format(location))}
    return figure
    
@app.callback(
    dash.dependencies.Output('output-container-range-slider', 'children'),
    [dash.dependencies.Input('slider', 'value')])
def update_output(value):
    dates=dict(zip(range(len(new_cases_t.index.unique())),
                   [pd.Timestamp(i).date().strftime('%d/%m') for i in new_cases_t.index]))
    
    return 'Dates selected from"{0}" to "{1}"'.format(dates[value[0]], dates[value[1]])


#app.run_server()

Para interactuar con los componentes, basta con generar un callback y una función que actualize las visualizaciones con los datos que entrega el componente

In [ ]:
app = dash.Dash()
app.layout=html.Div([html.Div([dcc.Graph(id='Map',
                              figure= {
                                  'data':[go.Choroplethmapbox(geojson=counties,
                                                              locations= new_cases.reset_index()['County'],
                                                              z=new_cases['2020-09-18'],
                                                              colorscale="Viridis", 
                                                              zmin=0, 
                                                              zmax=max(new_cases['2020-09-18']),
                                                              marker_opacity=0.5,
                                                              marker_line_width=0
                                                              )],
                                  'layout':go.Layout(mapbox_style='carto-darkmatter',
                                                    mapbox_zoom=5.75,
                                                    mapbox_center = {"lat": 35.782169, "lon": -80.793457},
                                                     hovermode= 'closest',
                                                    margin={"r":0,"t":0,"l":0,"b":0})
                              })], style={'width': '75%', 'display': 'inline-block'}
                               
                              ),
                     html.Div(dcc.RangeSlider(id= 'slider',
                                              updatemode = 'mouseup',
                                             min=0,
                                             max=len(new_cases_t.index.unique()),
                                             value=[0,
                                                    len(new_cases_t.index.unique())-1],
                                             )
                              
                              ),
                     html.Div(id='output-container-range-slider'),
                     
                     html.Div([dcc.Graph(id= 'TS',
                               figure= {
                                   'data': [go.Scatter(x=new_cases_t.index,
                                                       y=new_cases_t[new_cases_t.columns[0]],
                                                       visible=True)]}
                                        )])
                    ])


@app.callback(Output('TS', 'figure'),
             [Input('Map', 'clickData'),
             Input('slider', 'value')])
def callback_map(hoverData, value):
    location= hoverData['points'][0]['location']
    dates=dict(zip(range(len(new_cases_t.index.unique())),
               [pd.Timestamp(i).date() for i in new_cases_t.index]))
    from_date= dates[value[0]]
    to_date= dates[value[1]]
    filtered_data=new_cases_t[(new_cases_t.index>from_date) & (new_cases_t.index<to_date)]
    figure= {'data': [go.Scatter(x=filtered_data.index,
                                 y=filtered_data[location],
                                 visible=True)],
            'layout': go.Layout(title= '{0} new cases from {1} to {2}'.format(location, dates[value[0]],dates[value[1]]))}
    return figure
    
@app.callback(
    dash.dependencies.Output('output-container-range-slider', 'children'),
    [dash.dependencies.Input('slider', 'value')])
def update_output(value):
    dates=dict(zip(range(len(new_cases_t.index.unique())),
                   [pd.Timestamp(i).date().strftime('%d/%m') for i in new_cases_t.index]))
    
    return 'Dates selected from"{0}" to "{1}"'.format(dates[value[0]], dates[value[1]])

#app.run_server()

Es posible de esta manera, ligar un componente a mas de una visualización como también las visualizaciones entre ellas pudiendo generar visualizaciones altamente complejas. En este caso, ya se puede filtrar las series de tiempo con el mapa como también cambiar el mapa según los valores de tiempo del componente

In [ ]:
app = dash.Dash()
app.layout=html.Div([html.Div([dcc.Graph(id='Map',
                              figure= {
                                  'data':[go.Choroplethmapbox(geojson=counties,
                                                              locations= new_cases.reset_index()['County'],
                                                              z=new_cases_t.sum(),
                                                              colorscale="Viridis", 
                                                              zmin=0, 
                                                              zmax=max(new_cases_t.sum()),
                                                              marker_opacity=0.5,
                                                              marker_line_width=0
                                                              )],
                                  'layout':go.Layout(mapbox_style='carto-darkmatter',
                                                    mapbox_zoom=5.75,
                                                    mapbox_center = {"lat": 35.782169, "lon": -80.793457},
                                                    hovermode= 'closest',
                                                    margin={"r":0,"t":0,"l":0,"b":0})
                              })]
                               
                              ),
                     html.Div(dcc.RangeSlider(id= 'slider',
                                              updatemode = 'mouseup',
                                             min=0,
                                             max=len(new_cases_t.index.unique()),
                                             value=[0,
                                                    len(new_cases_t.index.unique())-1],
                                             )
                              
                              ),
                     html.Div(id='output-container-range-slider'),
                     
                     html.Div([dcc.Graph(id= 'TS',
                               figure= {
                                   'data': [go.Scatter(x=new_cases_t.index,
                                                       y=new_cases_t[new_cases_t.columns[0]],
                                                       visible=True)]}
                                        )])
                    ])


@app.callback(Output('TS', 'figure'),
             [Input('Map', 'clickData'),
             Input('slider', 'value')])
def callback_map(hoverData, value):
    location= hoverData['points'][0]['location']
    dates=dict(zip(range(len(new_cases_t.index.unique())),
               [pd.Timestamp(i).date() for i in new_cases_t.index]))
    from_date= dates[value[0]]
    to_date= dates[value[1]]
    filtered_data=new_cases_t[(new_cases_t.index>from_date) & (new_cases_t.index<to_date)]
    figure= {'data': [go.Scatter(x=filtered_data.index,
                                 y=filtered_data[location],
                                 visible=True)],
            'layout': go.Layout(title= '{0} new cases from {1} to {2}'.format(location, dates[value[0]],dates[value[1]]))}
    return figure

@app.callback(
    Output('Map', 'figure'),
    [Input('slider', 'value')])
def callback_map_color(value):
    dates=dict(zip(range(len(new_cases_t.index.unique())),
               [pd.Timestamp(i).date() for i in new_cases_t.index]))
    from_date= dates[value[0]]
    to_date= dates[value[1]]
    filtered_data=new_cases_t[(new_cases_t.index>from_date) & (new_cases_t.index<to_date)] 
    figure= {'data': [go.Choroplethmapbox(geojson=counties,
                                          locations= new_cases.reset_index()['County'],
                                          z=filtered_data.sum(),
                                          colorscale="Viridis", 
                                          zmin=min(filtered_data.sum()), 
                                          zmax=max(filtered_data.sum()),
                                          marker_opacity=0.5,
                                          marker_line_width=0
                                          )],
             
             'layout': go.Layout(mapbox_style='carto-darkmatter',
                                 mapbox_zoom=5.75,
                                 mapbox_center = {"lat": 35.782169, "lon": -80.793457},
                                 hovermode= 'closest',
                                 margin={"r":0,"t":0,"l":0,"b":0},
                                 )}
             
    return figure 

@app.callback(
    dash.dependencies.Output('output-container-range-slider', 'children'),
    [dash.dependencies.Input('slider', 'value')])
def update_output(value):
    dates=dict(zip(range(len(new_cases_t.index.unique())),
                   [pd.Timestamp(i).date().strftime('%d/%m') for i in new_cases_t.index]))
    
    return 'Dates selected from"{0}" to "{1}"'.format(dates[value[0]], dates[value[1]])

app.run_server()